In [4]:
import numpy as np
from typing import Union

In [11]:
"""
Given an integer B as represented by two arrays a, b
where B = the product over i of (a_i ^ b_i)
return the number of positive integers 
x less than B such that gcd(x, B) = 1
"""
def phi(a: np.array, b: np.array) -> int:
    
    result = 1
    
    for p,power in zip(a,b):
        
        result *= (p**power) * (1. - 1./p)
    
    return int(result)

In [12]:
a = [2,5]
b = [2,1]
phi(a,b)

8

In [30]:
"""
Given a factor base `factor_base` and an integer a, check whether or not 
a is B-smooth

returns a list `exponents` where each element 
exponents[i] is the exponent of the prime factor_base[i] 
if a is B-smooth

returns false otherwise
"""

def is_B_smooth(a: int, factor_base: np.array) -> Union[np.array, bool]:
    
    exponents = np.zeros_like(factor_base)
    
    for i,p in enumerate(factor_base):
        
        if a == 1:          
            break
        
        while a % p == 0:         
            a = a / p           
            exponents[i] += 1
        
    if a > 1:
        return False
    
    return exponents

In [36]:
factor_base = [2,3,5,7,11,13,17]
n = 539873
a = 783

is_B_smooth(a**2 % n, factor_base)

array([9, 0, 0, 0, 1, 1, 0])

In [16]:
'''
Given a vector of exponents, return the vector mod-2 
'''
def exponent_vector_mod_2(exponents: np.array) -> np.array:
    
    return exponents % 2

In [12]:
'''
Given an integer n, try to factor it into two large primes
Returns a tuple of the factors if able to factor
Returns false if couldn't factor
''' 
def quadratic_sieve(n: int) -> Union[tuple[int, int], bool]:
    
    
    """
    Choose smoothness bound B
    """
    
    """
    Represent B as product over i of (a_i^b_i)
    """
    
    """
    Find phi(B) + 1 numbers a_i such that 
        b_i = a_i^2 (mod n)
    is B-smooth.
    """
    
    n = phi(a,b)
    
    nums = []
    
    while len(nums) < n:
        
        continue
    
    """
    Factor the b_i and generate exponent vectors 
    (mod 2) for each b_i
    """
    
    """
    Find linear combination of these vectors that
    sums to the zero vector
    """
    
    """
    Multiply the corresponding a_i for the above 
    combination and call the result (mod n) 'a'
    
    Multiply the corresponding b_i for the above 
    combination and call the result (mod n) 'b'
    
    """
    
    
    
    
    pass